In [18]:
from deep_translator import GoogleTranslator
import json
import glob
import re
import os

import pandas as pd

path = "epir_train/"
# index = "Unnamed: 0"
articles = pd.read_csv(path + 'articles.csv', index_col="Unnamed: 0")

# life_situations = pd.read_csv(path + 'life_situations.csv', index_col="Unnamed: 0")
# news = pd.read_csv(path + 'news.csv', index_col="Unnamed: 0")
# services = pd.read_csv(path + 'services.csv', index_col="Unnamed: 0")

In [24]:
articles = articles.truncate(before=20000, after=20005)
# articles
def clean_df(df):
    text_cols = []
    for col in df.columns:
        if col not in ['id', 'sys_lang', 'subid', 'URL']:
            text_cols.append(col)

    df[text_cols] = df[text_cols].astype(str)
    mask = df[text_cols].apply(lambda x: x.str.split().str.len() < 10).all(axis=1)

    print("Rows that will be deleted:")
    print(df[mask])

    df = df.drop(df[mask].index)
    return df
articles = clean_df(articles)
def concat_text_columns(df, columns):
    df = df.copy()
    df[columns] = df[columns].replace('nan', '')
    df['text'] = df[columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    df = df.drop(columns=columns)
    return df
# articles = concat_text_columns(articles, ['title', 'content'])
print(articles)

Rows that will be deleted:
Empty DataFrame
Columns: [id, sys_lang, projects, url, text]
Index: []
          id sys_lang                           projects  \
20000  71240       kk                  kyzylorda-tabigat   
20001    940       kk                                bko   
20002  75517       ru      kostanai-karasy-audany-akimat   
20003  83645       kk                  kyzylorda-tabigat   
20004  61774       en  kostanai-altynsarin-audany-akimat   
20005  25698       ru                      aqmola-zhaksy   

                                                     url  \
20000  https://www.gov.kz/memleket/entities/kyzylorda...   
20001  https://www.gov.kz/memleket/entities/bko/press...   
20002  https://www.gov.kz/memleket/entities/kostanai-...   
20003  https://www.gov.kz/memleket/entities/kyzylorda...   
20004  https://www.gov.kz/memleket/entities/kostanai-...   
20005  https://www.gov.kz/memleket/entities/aqmola-zh...   

                                                    text  
2

In [25]:
articles.to_csv('data/articles.csv')

In [2]:
def split_string_into_chunks(text, max_chunk_size=4500):
    # Use regular expressions to split the text into sentences
    # Assumes sentences end with ".", "!", or "?", followed by a space or newline
    sentences = re.split(r'(?<=[.!?])\s+', text)
 
    # Initialize variables
    current_chunk = ""
    chunks = []
 
    # Iterate through sentences and add them to the current chunk
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            # If adding the sentence would exceed the chunk size, start a new chunk
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
 
    # Add the last chunk, if any
    if current_chunk:
        chunks.append(current_chunk.strip())
 
    return chunks

In [50]:
def translate(path):
    print(rf'{path}/*.csv')
    for file in glob.glob(rf'{path}/*.csv'):
        print(file)
        df = pd.read_csv(file, index_col="Unnamed: 0", engine="python")
        
        for ind, _ in df.iterrows():
            chunks = split_string_into_chunks(df.at[ind, "text"])
            df.at[ind, "text"] = ' '.join([GoogleTranslator(source='kk', target='en').translate(i) for i in chunks])
        

translate("data")

data/*.csv
data/articles.csv


RequestError: Request exception can happen due to an api connection error. Please check your connection and try again